# MNIST Digit Classification with our own Framework

Lab Assignment from [AI for Beginners Curriculum](https://github.com/microsoft/ai-for-beginners).

### Reading the Dataset

This code download the dataset from the repository on the internet. You can also manually copy the dataset from `/data` directory of AI Curriculum repo.

In [4]:
!rm *.pkl
!wget https://raw.githubusercontent.com/microsoft/AI-For-Beginners/main/data/mnist.pkl.gz
!gzip -d mnist.pkl.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  9.9M  100  9.9M    0     0   9.9M      0  0:00:01 --:--:--  0:00:01 15.8M


In [3]:
import pickle
with open('mnist.pkl','rb') as f:
    MNIST = pickle.load(f)

In [4]:
labels = MNIST['Train']['Labels']
data = MNIST['Train']['Features']

Let's see what is the shape of data that we have:

In [5]:
data.shape

(42000, 784)

### Splitting the Data

We will use Scikit Learn to split the data between training and test dataset:

In [6]:
from sklearn.model_selection import train_test_split

features_train, features_test, labels_train, labels_test = train_test_split(data,labels,test_size=0.2)

print(f"Train samples: {len(features_train)}, test samples: {len(features_test)}")

Train samples: 33600, test samples: 8400


### Instructions

1. Take the framework code from the lesson and paste it into this notebook, or (even better) into a separate Python module
1. Define and train one-layered perceptron, observing training and validation accuracy during training
1. Try to understand if overfitting took place, and adjust layer parameters to improve accuracy
1. Repeat previous steps for 2- and 3-layered perceptrons. Try to experiment with different activation functions between layers.
1. Try to answer the following questions:
    - Does the inter-layer activation function affect network performance?
    - Do we need 2- or 3-layered network for this task?
    - Did you experience any problems training the network? Especially as the number of layers increased.
    - How do weights of the network behave during training? You may plot max abs value of weights vs. epoch to understand the relation.

### Answer
1. Inter-layer activation function can both increase or decrease the network's performance. It may allow the network to break through soft peaks of the algorithm's accuracy, but it may also lead it into worse depths (without implementing backtracking)
2. By simply brute-forcing a few hyperparameters within the network's gradient descent, the accuracy of the network can increase drastically already. Adding more layers did not increase the accuracy of the network's predictions. Thus, adding aditional perceptron layers are not always the solution into learning a system.
3. As the number increase, the algorithm often converges to a set of weights as it deems to be the global max of the system when its only the local max. Further algorithms which allow the network to surpass local max can be implemented for possible higher accuracy.
4. 

In [1]:
import numpy as np
import pickle
import random
import gzip

def no_func(x):
    """No activation function."""
    return x

def relu(x):
    """ReLU activation function."""
    return np.maximum(0, x)

def softmax(x):
    """Softmax activation function."""
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

# Load the MNIST data with the correct encoding
with gzip.open('mnist.pkl.gz', 'rb') as mnist_pickle:
    MNIST = pickle.load(mnist_pickle, encoding='latin1')  # Specify encoding explicitly

training_data, validation_data, test_data = MNIST  # Unpack the tuple
# Access training features and labels
train_features, train_labels = training_data
val_features, val_labels = validation_data
test_features, test_labels = test_data
# Normalize features to the range [0, 1]
train_features = train_features.astype(np.float32) / 255.0
val_features = val_features.astype(np.float32) / 255.0
test_features = test_features.astype(np.float32) / 255.0

def one_hot_encode(labels, num_classes=10):
    """Convert integer labels to one-hot encoded vectors."""
    return np.eye(num_classes)[labels]

# Multi-layer Perceptron Weights Initialization
def initialize_mlp(layers):
    """
    Initialize weights and biases for a multi-layer perceptron.
    layers: List of layer sizes, e.g., [784, 128, 64, 10]
    Returns:
    - weights: List of weight matrices for each layer
    - biases: List of bias vectors for each layer
    """
    weights = []
    biases = []
    for i in range(len(layers) - 1):
        weights.append(np.random.randn(layers[i], layers[i + 1]) * 0.01)
        biases.append(np.zeros((1, layers[i + 1])))
    return weights, biases

# Forward pass through the network
def forward_mlp(X, weights, biases):
    """
    Perform a forward pass through the MLP.
    X: Input data (batch_size x input_size)
    weights: List of weight matrices
    biases: List of bias vectors
    activation_fn: Activation function to apply
    Returns:
    - activations: List of activations for each layer
    """
    activations = [X]
    for l in range(len(weights)):
        Z = np.dot(activations[-1], weights[l]) + biases[l]  # Weighted sum + bias
        # Apply ReLU for hidden layers, softmax for the output layer
        if l < len(weights) - 1:  # Hidden layers
            A = relu(Z)
        else:  # Output layer
            A = softmax(Z)
        activations.append(A)
    return activations

# Train function for a single layer
def train_layer(X, y, weights, biases, num_iterations, lambda_reg, learning_rate, activation_fn=relu):
    """
    Train a single layer of the MLP.
    X: Input features
    y: Target outputs
    weights, biases: Current weights and biases for the layer
    """
    for i in range(num_iterations):
        idx = random.randint(0, X.shape[0] - 1)
        x_sample = X[idx:idx + 1]  # Random single sample
        y_sample = y[idx:idx + 1]  # Corresponding label

        # Forward pass through the layer
        Z = np.dot(x_sample, weights) + biases
        A = activation_fn(Z)

        # Compute error (difference between prediction and target)
        error = A - y_sample

        # Update weights and biases using the error
        weights -= learning_rate * np.dot(x_sample.T, error) + lambda_reg * weights
        biases -= learning_rate * error

        # # Debug: Print weight updates
        # if i % 10 == 0:  # Every 10 iterations
        #     print(f"Iteration {i}, Weights Norm: {np.linalg.norm(weights):.4f}, Error Norm: {np.linalg.norm(error):.4f}")

    return weights, biases

# Train the entire network, simultaneously training hidden layers
def train_mlp(train_features, train_labels, layers, num_iterations, lambda_reg, learning_rate):
    weights, biases = initialize_mlp(layers)

    for i in range(num_iterations):
        # Forward pass through all layers
        activations = forward_mlp(train_features, weights, biases)
        
        # Compute error at the output layer
        error = activations[-1] - train_labels
        
        # Update weights and biases for the output layer
        weights[-1] -= learning_rate * np.dot(activations[-2].T, error) + lambda_reg * weights[-1]
        biases[-1] -= learning_rate * np.mean(error, axis=0, keepdims=True)
        
        # # Debugging: Print loss every 10 iterations
        # if i % 10 == 0:
        #     loss = -np.mean(train_labels * np.log(activations[-1] + 1e-9))  # Cross-entropy loss
        #     print(f"Iteration {i}, Loss: {loss:.4f}")
    
    return weights, biases


# Classification through the network
def classify_mlp(test_features, weights, biases):
    """
    Classify samples using the trained MLP.
    test_features: Input features
    weights, biases: Trained weights and biases
    """
    activations = forward_mlp(test_features, weights, biases)
    output_layer = activations[-1]  # Softmax output from the final layer
    
    # Predicted class is the one with the highest probability
    predicted_labels = np.argmax(output_layer, axis=1)
    print("First 5 predictions (probabilities):", output_layer[:5])
    return predicted_labels

In [2]:
# View activation functions here: https://www.geeksforgeeks.org/activation-functions-neural-networks/

# Define the architecture
layers = [784, 256, 64, 10]  # Example: 2 hidden layers with 128 and 64 neurons

# One-hot encode the labels for training
train_labels_one_hot = one_hot_encode(train_labels)

# Train the MLP
weights, biases = train_mlp(
    train_features,
    train_labels_one_hot,
    layers,
    num_iterations=200,
    lambda_reg=0.01,
    learning_rate=0.1,
)

# Test the MLP
predicted_labels = classify_mlp(test_features, weights, biases)
accuracy = np.mean(predicted_labels == test_labels)
print(f"Test accuracy: {accuracy}")

First 5 predictions (probabilities): [[0.09877945 0.11195301 0.09939602 0.1016998  0.09752218 0.09155338
  0.0991078  0.10301774 0.09722568 0.09974495]
 [0.09879936 0.11193573 0.09941009 0.10174123 0.09750016 0.09156223
  0.09910821 0.10296829 0.09725476 0.09971994]
 [0.09876133 0.11198903 0.09939948 0.10170149 0.0975277  0.09154807
  0.09910725 0.1030021  0.09721723 0.09974631]
 [0.0988173  0.11192094 0.09940208 0.10170825 0.09751097 0.09155885
  0.09911603 0.10298894 0.09724541 0.09973122]
 [0.09878481 0.11194863 0.09939463 0.10169948 0.09752915 0.09155167
  0.09910112 0.10300623 0.0972332  0.09975109]]
Test accuracy: 0.1135
